In [1]:
from Bio import Entrez
Entrez.email = "nicholas.underhill@sky.com"
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import shutil
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
import math
from tqdm import tqdm
from Comparative_Analysis import Utilities as util
from joblib import Parallel, delayed
from Comparative_Analysis import Blast_Functions as blastfn
import time
import os
import copy
import collections
from scipy.stats import chi2, binom
from Comparative_Analysis import Alignment as alignfn
from Bio.Align.Applications import MuscleCommandline
import subprocess
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
project_dir = 'F:/Project_Data/mabR_Project/MTBC_Strains'
seq_dir = 'F:/Datasets/Other_Genomes/MTBC_Strains'
motif_loc = 'F:/Project_Data/mabR_Project/Second_MEME_Upstream_Output/meme.txt'
full_run = True
muscle_exe = 'C:/Users/nicho/Muscle/muscle3.8.31_i86win32.exe'

In [28]:
species_names = []
tbc_species_dirs = util.list_dirs(seq_dir)
for n, sdir in tqdm(enumerate(tbc_species_dirs)):
    fname = util.list_files(seq_dir + '/' + sdir)[0]
    with open(seq_dir + '/' + sdir + '/' + fname,'r') as ofile: 
        for l in ofile:
            species_names.append(l.split(' ')[0][1:])
            break
acc_ids = '"' + ','.join(species_names[0:100]) + '"'
print(acc_ids)

371it [00:01, 260.51it/s]

"NZ_OW052570.1,NC_002755.2,NC_008769.1,NC_012207.1,NC_009525.1,NC_009565.1,NC_012943.1,NC_022350.1,NC_016768.1,NC_018078.1,NZ_CP012090.1,NZ_CP003494.1,NC_002945.4,NC_000962.3,NC_017524.1,NC_016804.1,NC_015758.1,NC_015848.1,NC_017523.1,NC_017522.1,NC_018143.2,NC_020089.1,NC_020245.2,NC_020559.1,NC_021192.1,NC_021193.1,NC_021194.1,NC_021251.1,NC_021740.1,NZ_CP002871.1,NZ_CP002882.1,NZ_CP002883.1,NZ_CP002885.1,NZ_CP007803.1,NZ_CP007809.1,NZ_CP009100.1,NZ_CP009101.1,NZ_CP009426.1,NZ_CP009427.1,NZ_HG813240.1,NZ_CP007027.1,NZ_AP014573.1,NZ_CP015773.2,NZ_CP010873.1,NZ_AM412059.1,NZ_CP008744.1,NZ_CP012095.1,NZ_CP009243.1,NZ_CP012506.2,NZ_CP013741.1,NZ_CP010329.1,NZ_CP010330.1,NZ_CP010335.1,NZ_CP010336.1,NZ_CP010337.1,NZ_CP010338.1,NZ_CP010339.1,NZ_CP010340.1,NZ_CP014566.1,NZ_CP046529.1,NZ_CP016794.1,NZ_CP016888.1,NZ_CP011510.1,NZ_CP013475.1,NZ_CP017920.1,NZ_CP018303.1,NZ_CP018305.1,NZ_CP018302.1,NZ_CP018301.1,NZ_CP018300.1,NZ_CP018304.1,NZ_CP018778.1,NZ_CP016972.1,NZ_CP020381.2,NZ_CP017593.1,N

In [23]:
acc_ids = "NC_000962.3"

In [29]:
with Entrez.efetch(db="nucleotide",id=acc_ids, rettype = 'gb', retmode = 'text') as handle:
    for seq_record in SeqIO.parse(handle, "gb"):
        accession_ver = seq_record.annotations['accessions'][0] + '.' + str(seq_record.annotations['sequence_version'])
        organism = seq_record.annotations['organism']
        print(accession_ver, organism, seq_record.annotations)

NC_002755.2 Mycobacterium tuberculosis CDC1551 {'molecule_type': 'DNA', 'topology': 'circular', 'data_file_division': 'CON', 'date': '07-JUN-2023', 'accessions': ['NC_002755', 'NZ_AE006915-NZ_AE007194'], 'sequence_version': 2, 'keywords': ['RefSeq'], 'source': 'Mycobacterium tuberculosis CDC1551', 'organism': 'Mycobacterium tuberculosis CDC1551', 'taxonomy': ['Bacteria', 'Actinomycetota', 'Actinomycetes', 'Mycobacteriales', 'Mycobacteriaceae', 'Mycobacterium', 'Mycobacterium tuberculosis complex'], 'references': [Reference(title='Whole-genome comparison of Mycobacterium tuberculosis clinical and laboratory strains', ...), Reference(title='Direct Submission', ...), Reference(title='Direct Submission', ...)], 'comment': 'REFSEQ INFORMATION: The reference sequence is identical to\nAE000516.2.\nOn Aug 4, 2004 this sequence version replaced NC_002755.1.\nThe annotation was added by the NCBI Prokaryotic Genome Annotation\nPipeline (PGAP). Information about PGAP can be found here:\nhttps://ww

KeyboardInterrupt: 